In [ ]:
import copy

import numpy as np
from matplotlib import pyplot as plt
#Import the needed opentps.core packages
import time
import os
import logging
import math
import numpy as np
from matplotlib import pyplot as plt
import sys
from opentps.core.io.dicomIO import writeRTPlan
from opentps.core.processing.planOptimization.tools import evaluateClinical
from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.data.plan import FidObjective
from opentps.core.io.scannerReader import readScanner
from opentps.core.io.serializedObjectIO import saveRTPlan, loadRTPlan
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D
from opentps.core.processing.C_libraries.libRayTracing_wrapper import WET_raytracing
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.io import mcsquareIO
from opentps.core.io.mcsquareIO import _dicomIsocenterToMCsquare
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.io.dataLoader import readData
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import FidObjective
from opentps.core.io.serializedObjectIO import saveBeamlets, saveRTPlan, loadBeamlets, loadRTPlan

############################################### 0604 TG119 TEST ##############################################
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL(DoseCalculationConfig().bdlFile)
ctImagePath = "E:\OpenTPS\opentps-master\opentps_core\opentps\core\examples\planOptimization\AOCHANGJIAN"
data = readData(ctImagePath)
print(data[0])
# RD RT RS CT
# RD RD RD RD RD RP RS CT


09/04/2025 02:40:33 PM - root - INFO - INFO: cannot recognize file format of E:\OpenTPS\opentps-master\opentps_core\opentps\core\examples\planOptimization\AOCHANGJIAN\Thumbs.db
09/04/2025 02:40:33 PM - root - INFO - INFO: cannot recognize file format of E:\OpenTPS\opentps-master\opentps_core\opentps\core\examples\planOptimization\AOCHANGJIAN\Thumbs.db
Image3D 98 x 117 x 113



In [3]:
print(data)

[<opentps.core.data.images._doseImage.DoseImage object at 0x000001A90436EEE0>, <opentps.core.data.images._doseImage.DoseImage object at 0x000001A90527DEE0>, <opentps.core.data.images._doseImage.DoseImage object at 0x000001A905287EB0>, <opentps.core.data.images._doseImage.DoseImage object at 0x000001A90528FE80>, <opentps.core.data.images._doseImage.DoseImage object at 0x000001A905299E50>, <opentps.core.data.images._doseImage.DoseImage object at 0x000001A9052A2E20>, <opentps.core.data.plan._rtPlan.RTPlan object at 0x000001A90528A0D0>, <opentps.core.data._rtStruct.RTStruct object at 0x000001A904409F10>, <opentps.core.data.images._ctImage.CTImage object at 0x000001A905CD34F0>]


In [4]:
plan=data[-3]
rt_struct = data[-2]
ct = data[-1]
rt_struct.print_ROINames()
print("CT spacing=",ct.spacing)
print("CT grid size",ct.gridSize)
target_name = "PTV"



RT Struct UID: 1.2.752.243.1.1.20240805160157479.8700.15173.1
  [0]  GTV
  [1]  Lens_L
  [2]  Lens_R
  [3]  Temporal_Lobe_L
  [4]  Temporal_Lobe_R
  [5]  Brain_Stem
  [6]  Brain_Stem_PRV
  [7]  PTV
  [8]  Lens_L_PRV
  [9]  Lens_R_PRV
  [10]  Lens_all_PRV
  [11]  BODY
  [12]  NTPTV
  [13]  External
  [14]  PTV_plan
  [15]  Ring1PTV
  [16]  Ring2PTV
  [17]  Ring3PTV
  [18]  Ring4PTV
  [19]  Fan
  [20]  MID
  [21]  3000
  [22]  3001
  [23]  5700
  [24]  30
  [25]  Brain
  [26]  brian-PTV
  [27]  60
  [28]  30-3pr
  [29]  60-3pr
  [30]  30_3Robust
  [31]  60_3Robust
  [32]  30_5Robust
  [33]  60_5Robust
  [34]  locref
  [35]  iso
CT spacing= [0.976563 0.976563 1.      ]
CT grid size [512 512 222]


In [5]:
#MCsquare simulation
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration
mc2.nbPrimaries = 1e7
doseImage = mc2.computeDose(ct, plan)

09/04/2025 02:40:36 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
09/04/2025 02:40:36 PM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
09/04/2025 02:40:36 PM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\PlanPencil.txt
09/04/2025 02:40:37 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


In [6]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import numpy as np
def plot_Zslice(z_index):
    #plt.clf()
    plt.figure(figsize=(8, 6))
    x_size=doseImage.imageArray.shape[0]
    y_size=doseImage.imageArray.shape[1]
    X=np.linspace(0,x_size-1,x_size)
    Y=np.linspace(0,y_size-1,y_size)
    X, Y = np.meshgrid(X, Y)
    #plt.contourf(X, Y, covered_count[:,:,z_index], cmap='jet',levels=10,vmin=0,vmax=np.max(covered_count),alpha=0.6)
    plt.imshow(doseImage.imageArray[:,:,z_index].transpose(1,0), cmap='jet', interpolation='nearest', alpha=0.6)
    plt.colorbar()
    #绘制边界
    #plt.contour(X, Y, expanded_image[:,:,z_index], colors='k',linewidths=2,levels=[0.5])
    plt.plot()
    plt.show()   
interact(plot_Zslice, z_index=IntSlider(min=0, max=doseImage.imageArray.shape[2]-1, step=1))     

interactive(children=(IntSlider(value=0, description='z_index', max=221), Output()), _dom_classes=('widget-int…

<function __main__.plot_Zslice(z_index)>

In [ ]:
from opentps.core.io.dicomIO import writeRTDose
output_folder = os.getcwd()
writeRTDose(plan,output_folder,"opentps_SFRT_250116.dcm")

# import pydicom
# import datetime
# # Don't delete it
# # ++++Don't delete UIDs to build the simple study+++++++++++++++++++
# studyInstanceUID = pydicom.uid.generate_uid()
# doseSeriesInstanceUID = pydicom.uid.generate_uid()
# planSeriesInstanceUID = pydicom.uid.generate_uid()
# ctSeriesInstanceUID =  pydicom.uid.generate_uid()
# frameOfReferenceUID = pydicom.uid.generate_uid()
# # structSeriesInstanceUID = pydicom.uid.generate_uid()
# dt = datetime.datetime.now()
# # CT
# patient = Patient()
# patient.name = 'Simple_Patient'
# Patient.id = 'Simple_Patient'
# Patient.birthDate = dt.strftime('%Y%m%d')
# patient.sex = ""

# doseImage.referencePlan = plan
# doseImage.referenceCT = ct
# doseImage.patient = patient
# doseImage.studyInstanceUID = studyInstanceUID
# doseImage.frameOfReferenceUID = frameOfReferenceUID 
# doseImage.sopClassUID = '1.2.840.10008.5.1.4.1.1.481.2'
# doseImage.mediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.481.2'
# doseImage.sopInstanceUID = pydicom.uid.generate_uid()
# doseImage.studyTime = dt.strftime('%H%M%S.%f')
# doseImage.studyDate = dt.strftime('%Y%m%d')

# writeRTDose(doseImage,output_folder)

09/04/2025 02:45:43 PM - opentps.core.io.dicomIO - INFO - Export dicom RTDOSE: RD1.2.826.0.1.3680043.8.498.13788487360005424824957731664625185789.dcm in e:\opentps-v1.1.1\opentps_core\opentps\core\examples\planOptimization


d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'SoftwareVersion' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'OperatorName' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'Width' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\valuerep.py:443: UserWarning: A value of type 'int32' cannot be assigned to a tag with VR US.
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'Height' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packag